# Analyzing Traffic Patterns to Wikimedia Projects

In [1]:
import org.apache.log4j._
Logger.getRootLogger().setLevel(Level.ERROR)

This data dump is taken from Wikimedia site : https://dumps.wikimedia.org/other/pageviews/2017/

* It holds daily consolidated traffic data based on the article requests.

* Data is found is space separated colums in human readable format

`
Note: preprocessing is done to filter out only EN articles and saved them in parquet format
`


### Task for Data Analyst

### Load/Read Data

In [34]:
val df = spark.read.parquet("/Users/josiahsams/SparkDemo/datasets/parquet/pageviews")

df: org.apache.spark.sql.DataFrame = [project: string, article: string ... 1 more field]


In [3]:
df.printSchema

root
 |-- project: string (nullable = true)
 |-- article: string (nullable = true)
 |-- requests: integer (nullable = true)



In [4]:
df.describe().show

+-------+-----------------+
|summary|         requests|
+-------+-----------------+
|  count|          2095763|
|   mean|4.252991869786803|
| stddev|659.0505003693299|
|    min|                1|
|    max|           948323|
+-------+-----------------+



In [5]:
df.rdd.getNumPartitions

res3: Int = 4


In [6]:
df.show

+-------+--------------------+--------+
|project|             article|requests|
+-------+--------------------+--------+
|     en|                 !!!|       4|
|     en|       !Ora_language|       1|
|     en|    "A"_Is_for_Alibi|       1|
|     en|    Awaken,_My_Love!|      14|
|     en|"C"_Is_for_(Pleas...|       2|
|     en|   Come_On,_Let's_Go|       1|
|     en|  "Dear_Boss"_letter|       1|
|     en|  "G"_Is_for_Gumshoe|       1|
|     en|              Heroes|       1|
|     en|       "I_want"_song|       1|
|     en|   "M"_Is_for_Malice|       1|
|     en|"Panzer_ace"_in_p...|       1|
|     en| "R"_Is_for_Ricochet|       1|
|     en|     "Sin"_Blackwood|       1|
|     en|"This_Is_Our_Punk...|       1|
|     en|          "V"_device|      29|
|     en|"Weird_Al"_Yankov...|      12|
|     en|  "Wild_Bill"_Hickok|       9|
|     en|                   $|       1|
|     en|               $1000|       1|
+-------+--------------------+--------+
only showing top 20 rows



In [7]:
import org.apache.spark.sql.functions._

### List down different type of projects available in EN

In [35]:
df.groupBy("project").count().orderBy(col("count").desc).show()

+---------+-------+
|  project|  count|
+---------+-------+
|       en|1185441|
|     en.m| 783366|
|  en.zero|  51727|
|     en.d|  36218|
|   en.m.d|  19915|
|     en.b|   4472|
|     en.q|   2712|
|   en.m.b|   2669|
|     en.s|   1873|
|   en.m.q|   1771|
|   en.voy|   1295|
|     en.v|    988|
|   en.m.s|    912|
|en.zero.d|    512|
| en.m.voy|    510|
|   en.m.v|    400|
|     en.n|    362|
|en.zero.b|    274|
|   en.m.n|     99|
|en.zero.q|     83|
+---------+-------+
only showing top 20 rows



### Filter out only EN wikipedia articles

In [9]:
val filterEN = df.filter(col("project") === "en").cache()


filterEN: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [project: string, article: string ... 1 more field]


### Get Top Articles based on the requests

In [10]:
filterEN.orderBy(col("requests").desc).show

+-------+--------------------+--------+
|project|             article|requests|
+-------+--------------------+--------+
|     en|           Main_Page|  948323|
|     en|      Special:Search|   48502|
|     en|                   -|    7662|
|     en|List_of_macOS_com...|    6561|
|     en|Star_Trek:_Discovery|    4974|
|     en|        Special:Book|    4464|
|     en|German_federal_el...|    3818|
|     en|         Darth_Vader|    3749|
|     en|  Special:LinkSearch|    3125|
|     en|Alternative_for_G...|    3040|
|     en|      Deaths_in_2017|    2453|
|     en|        Frauke_Petry|    2249|
|     en|     No_Mercy_(2017)|    2208|
|     en|Catherine_Zeta-Jones|    2096|
|     en|   Special:Watchlist|    2040|
|     en| Special:ElectronPdf|    2028|
|     en|       Howard_Hughes|    1899|
|     en|               Earth|    1847|
|     en|Kingsman:_The_Gol...|    1750|
|     en|      It_(2017_film)|    1705|
+-------+--------------------+--------+
only showing top 20 rows



In [11]:
val filterDF = filterEN
  .filter(col("article").rlike("""^(?!-)"""))
  .filter(col("article").rlike("""^((?!Special:)+)"""))
  .filter(col("article").rlike("""^((?!File:)+)"""))
  .filter(col("article").rlike("""^((?!Category:)+)"""))
  .filter(col("article").rlike("""^((?!User:)+)"""))
  .filter(col("article").rlike("""^((?!Talk:)+)"""))
  .filter(col("article").rlike("""^((?!Template:)+)"""))
  .filter(col("article").rlike("""^((?!Help:)+)"""))
  .filter(col("article").rlike("""^((?!Wikipedia:)+)"""))
  .filter(col("article").rlike("""^((?!MediaWiki:)+)"""))
  .filter(col("article").rlike("""^((?!Portal:)+)"""))
  .filter(col("article").rlike("""^((?!Book:)+)"""))
  .filter(col("article").rlike("""^((?!Draft:)+)"""))
  .filter(col("article").rlike("""^((?!Education_Program:)+)"""))
  .filter(col("article").rlike("""^((?!TimedText:)+)"""))
  .filter(col("article").rlike("""^((?!Module:)+)"""))
  .filter(col("article").rlike("""^((?!Topic:)+)"""))
  .filter(col("article").rlike("""^((?!Images/)+)"""))
  .filter(col("article").rlike("""^((?!%22//upload.wikimedia.org)+)"""))
  .filter(col("article").rlike("""^((?!%22//en.wikipedia.org)+)"""))
val cachedDF = filterDF.cache()

filterDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [project: string, article: string ... 1 more field]
cachedDF: filterDF.type = [project: string, article: string ... 1 more field]


In [12]:
cachedDF.orderBy(col("requests").desc).select("article", "requests").show

+--------------------+--------+
|             article|requests|
+--------------------+--------+
|           Main_Page|  948323|
|List_of_macOS_com...|    6561|
|Star_Trek:_Discovery|    4974|
|German_federal_el...|    3818|
|         Darth_Vader|    3749|
|Alternative_for_G...|    3040|
|      Deaths_in_2017|    2453|
|        Frauke_Petry|    2249|
|     No_Mercy_(2017)|    2208|
|Catherine_Zeta-Jones|    2096|
|       Howard_Hughes|    1899|
|               Earth|    1847|
|Kingsman:_The_Gol...|    1750|
|      It_(2017_film)|    1705|
|Rick_and_Morty_(s...|    1622|
|          Forum_spam|    1516|
|            Al-Batin|    1433|
|    Colin_Kaepernick|    1401|
|        Alice_Weidel|    1266|
|        September_25|    1194|
+--------------------+--------+
only showing top 20 rows



### How many requests did the "Apache Spark" article recieve during this hour?

In [13]:
cachedDF.filter(col("article") === "Apache_Spark").select("requests").show()

+--------+
|requests|
+--------+
|      53|
+--------+



### Which Apache project received the most requests during this hour?

In [14]:
cachedDF.filter($"article".startsWith("Apache_"))
     .select("article", "requests")
     .orderBy(col("requests").desc)
     .show()

+--------------------+--------+
|             article|requests|
+--------------------+--------+
|       Apache_Tomcat|     131|
|       Apache_Hadoop|     100|
|   Apache_OpenOffice|      85|
|  Apache_HTTP_Server|      54|
|        Apache_Spark|      53|
|        Apache_Kafka|      45|
|    Apache_Cassandra|      43|
|      Apache_Parquet|      30|
|      Apache_License|      29|
|        Apache_Maven|      29|
|     Apache_Struts_2|      26|
|   Apache_Subversion|      23|
|      Apache_Cordova|      23|
|         Apache_Hive|      20|
|         Apache_Solr|      17|
|        Apache_Mesos|      17|
|Apache_Software_F...|      17|
|    Apache_ZooKeeper|      16|
|          Apache_Ant|      15|
|        Apache_HBase|      13|
+--------------------+--------+
only showing top 20 rows



### More SQL Queries: Functions

In [6]:
import org.apache.spark.sql.functions._

In [1]:
spark.sqlContext.setConf("spark.sql.shuffle.partitions", "3")

In [32]:
val pageviewsDF = spark.read.parquet("/Users/josiahsams/SparkDemo/datasets/repart-pageviews-by-second-tsv.parquet").cache

pageviewsDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [timestamp: timestamp, site: string ... 1 more field]


In [11]:
pageviewsDF.printSchema

root
 |-- timestamp: timestamp (nullable = true)
 |-- site: string (nullable = true)
 |-- requests: integer (nullable = true)



In [54]:
pageviewsDF.show

+--------------------+-------+--------+
|           timestamp|   site|requests|
+--------------------+-------+--------+
|2015-03-16 02:20:...|desktop|    2350|
|2015-03-16 04:55:...|desktop|    2211|
|2015-03-16 07:47:...| mobile|     878|
|2015-03-16 10:39:...|desktop|    2391|
|2015-03-16 13:39:...|desktop|    2943|
|2015-03-16 16:01:...| mobile|    1438|
|2015-03-16 18:49:...|desktop|    3239|
|2015-03-16 22:08:...|desktop|    2476|
|2015-03-16 00:31:...| mobile|    1586|
|2015-03-16 03:46:...| mobile|    1535|
|2015-03-16 06:46:...| mobile|     959|
|2015-03-16 09:49:...| mobile|     943|
|2015-03-16 12:32:...| mobile|    1028|
|2015-03-16 15:03:...| mobile|    1298|
|2015-03-16 17:48:...|desktop|    3235|
|2015-03-16 20:36:...| mobile|    1430|
|2015-03-16 22:58:...| mobile|    1402|
|2015-03-16 01:26:...|desktop|    2427|
|2015-03-16 04:01:...| mobile|    1480|
|2015-03-16 06:52:...|desktop|    2191|
+--------------------+-------+--------+
only showing top 20 rows



### Get all the requests coming from mobile to Wikimedia

In [10]:
pageviewsDF.filter($"site" === "mobile")
    .select(sum($"requests")).show

+-------------+
|sum(requests)|
+-------------+
|   4605793167|
+-------------+



### List the months on which this dataset is collected

In [14]:
pageviewsDF.select(month($"timestamp").alias("Months")).distinct().show

+------+
|Months|
+------+
|     3|
|     4|
+------+



Functions like min, max, avg [more](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.functions$) .. can also be used  

### Which day of the week gets most traffic

In [20]:
val unorderedList = pageviewsDF.groupBy(date_format($"timestamp", "E").alias("Day_of_Week"))
            .sum().cache
unorderedList.show

unorderedList: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Day_of_Week: string, sum(requests): bigint]


+-----------+-------------+
|Day_of_Week|sum(requests)|
+-----------+-------------+
|        Tue|   1995034884|
|        Thu|   1931508977|
|        Sun|   1576726066|
|        Mon|   2356809353|
|        Wed|   1977615396|
|        Fri|   1842512718|
|        Sat|   1662762048|
+-----------+-------------+



In [21]:
val orderByReq = unorderedList.orderBy($"sum(requests)".desc)
orderByReq.show

orderByReq: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Day_of_Week: string, sum(requests): bigint]


+-----------+-------------+
|Day_of_Week|sum(requests)|
+-----------+-------------+
|        Mon|   2356809353|
|        Tue|   1995034884|
|        Wed|   1977615396|
|        Thu|   1931508977|
|        Fri|   1842512718|
|        Sat|   1662762048|
|        Sun|   1576726066|
+-----------+-------------+



### User defined Function

In [26]:
def prependNumberToDay(day: String) = {
    day match {
        case "Sun" => "1-Sun"
        case "Mon" => "2-Mon"
        case "Tue" => "3-Tue"
        case "Wed" => "4-Wed"
        case "Thu" => "5-Thu"
        case "Fri" => "6-Fri"
        case "Sat" => "7-Sat"
        case _ => "UNKNOWN"
  }
}

prependNumberToDay: (day: String)String


In [61]:
val prependNumUDF = spark.sqlContext.udf.register("prependNum", (s: String) => prependNumberToDay(s))

prependNumUDF: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function1>,StringType,Some(List(StringType)))


In [52]:
orderByReq.select(prependNumUDF($"Day_of_Week").alias("Day"), $"sum(requests)" ).show

+-----+-------------+
|  Day|sum(requests)|
+-----+-------------+
|2-Mon|   2356809353|
|3-Tue|   1995034884|
|4-Wed|   1977615396|
|5-Thu|   1931508977|
|6-Fri|   1842512718|
|7-Sat|   1662762048|
|1-Sun|   1576726066|
+-----+-------------+



In [40]:
%lsmagic

res23: org.apache.toree.magic.MagicOutput = MagicOutput(List())


Available line magics:
%lsmagic %showtypes %adddeps %truncation %addjar

Available cell magics:
%%sql %%html %%javascript %%dataframe %%pyspark %%scala %%sparkr

Type %<magic_name> for usage info.
         


### SQL queries

In [43]:
pageviewsDF.createOrReplaceTempView("pageViewTable")

Get all the requests coming from mobile to Wikimedia

In [49]:
%ShowTypes off

res30: org.apache.toree.magic.MagicOutput = MagicOutput(List())


Types will not be printed


In [50]:
%%sql select sum(requests) from pageViewTable where site = "mobile"

res31: org.apache.toree.magic.MagicOutput =
MagicOutput(ArrayBuffer((text/plain,+-------------+
|sum(requests)|
+-------------+
|   4605793167|
+-------------+
)))


In [59]:
%%sql 
desc pageViewTable

res42: org.apache.toree.magic.MagicOutput =
MagicOutput(ArrayBuffer((text/plain,+---------+---------+-------+
| col_name|data_type|comment|
+---------+---------+-------+
|timestamp|timestamp|   null|
|     site|   string|   null|
| requests|      int|   null|
+---------+---------+-------+
)))
